# Przygotowanie środowiska

In [1]:
%matplotlib inline 

import cv2
import numpy as np
import os
import pickle
import sys
import random
import tensorflow as tf
import matplotlib.pyplot as plt
import math
import keras
import functools 
import datetime

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn import svm

from skimage import exposure, feature

from keras.models import Sequential, Model, load_model
from keras.layers import Dense, Dropout, Activation, Conv2D, MaxPooling2D, Flatten, GlobalAveragePooling2D
from keras.utils import np_utils
from keras.applications.vgg19 import VGG19, preprocess_input
from keras.preprocessing import image
from keras.optimizers import SGD, Adam
from scipy.ndimage.interpolation import rotate, shift

C:\Users\adama\AppData\Local\conda\conda\envs\stable\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def top_k(n):
    top_k_acc = functools.partial(keras.metrics.top_k_categorical_accuracy, k=n)
    top_k_acc.__name__ = 'top_'+str(n)
    return top_k_acc

In [3]:
def cmc(model, test_set, d_3, SVM):    
    if SVM is not None:
        with tf.Session() as sess:
            cmc_curves = [0]
            for i in range(1, class_count+1):
                cmc_curves.append(
                    top_k(i)(
                        np_utils.to_categorical(test_set[:, 1], num_classes=class_count), 
                        SVM.astype(np.float32)
                    ).eval()
                )
    else:
        model.compile(optimizer=SGD(lr=0.0005, momentum=0.9),
              loss='categorical_crossentropy',
              metrics=['accuracy']+[top_k(k) for k in range(2, class_count + 1)])
        if d_3:
            cmc_curves = model.evaluate(np.stack(test_set[:, 0], axis = 0).reshape((test_set.shape[0], size_img, size_img, 3)), 
                           np_utils.to_categorical(test_set[:, 1], num_classes=class_count),
                           batch_size=32,
                           verbose = 1)
        else:
            cmc_curves = model.evaluate(np.stack(test_set[:, 0], axis = 0), 
                           np_utils.to_categorical(test_set[:, 1], num_classes=class_count),
                           batch_size=32,
                           verbose = 1) 
        
    cmc_curves = np.array([0] + cmc_curves[1:])*100
    plt.xlabel('k')
    plt.ylabel('Dokładność Top-k [%]')
    plt.plot(range(class_count + 1), np.array(range(class_count + 1))*100/(class_count), color='green', linestyle='--')
    plt.plot(range(class_count + 1), cmc_curves)
    plt.scatter([1, 5], [cmc_curves[1], cmc_curves[5]])
    plt.grid()
    plt.show()
    print("Dokładność TOP-1:", '%0.2f' % cmc_curves[1], end="%\n")
    print("Dokładność TOP-5:", '%0.2f' % cmc_curves[5], end="%\n")
    return cmc_curves

def more_cmc(cmc_set, names, leg_title, filename):
    plt.xlabel('k')
    plt.ylabel('Dokładność Top-k [%]')
    plt.plot(range(class_count + 1), np.array(range(class_count + 1))*100/(class_count), color='green', linestyle='--')
    for curve, name in zip(cmc_set, names):
        plt.plot(range(class_count + 1), curve, label=name)
        plt.scatter([1, 5], [curve[1], curve[5]])
        plt.legend(loc = 'lower right', title = leg_title)
    plt.grid()
    if filename is not None:
        plt.savefig(filename, dpi = 200)
    plt.show()

In [4]:
def history(hist1, hist2):
    plt.xlabel('Numer epoki')
    plt.ylabel('Dokładność na zbiorze trenującym [%]')
    epochs = len(hist1.epoch) + len(hist2.epoch)
    
    plt.plot(range(1, epochs+1), np.array(hist1.history['acc'] + hist2.history['acc'])*100, 
             range(1, epochs+1), np.array(hist1.history['top_5'] + hist2.history['top_5'])*100)
    plt.grid()
    plt.show()
    print("Historia uczenia:")
    print("\tEpoka:\t", np.array(range(1, epochs+1)))
    print("\tTop-1:\t", (np.array(hist1.history['acc'] + hist2.history['acc'])*100))
    print("\tTop-5:\t", (np.array(hist1.history['top_5'] + hist2.history['top_5'])*100))
    
def more_history(history_set_1, history_set_2, t, names, leg_title, filename):
    plt.xlabel('Numer epoki')
    plt.ylabel('Dokładność na zbiorze trenującym [%]')
    
    for hist_1, hist_2, name in zip(history_set_1, history_set_2, names):
        epochs = len(hist_1.epoch) + len(hist_2.epoch)
        plt.plot(range(1, epochs+1), np.array(hist_1.history[t] + hist_2.history[t])*100, label=name)
        plt.legend(loc = 'lower right', title = leg_title)

    if len(history_set_1) > len(history_set_2):
        for hist, name in zip(history_set_1[len(history_set_2):], names[len(history_set_2):]):
            epochs = len(hist.epoch)
            plt.plot(range(1, epochs+1), np.array(hist.history[t])*100, label=name)
            plt.legend(loc = 'lower right', title = leg_title)

    plt.grid()
    if filename is not None:
        plt.savefig(filename, dpi = 200)
    plt.show()

# Skrypt sterujący

In [5]:
folder_path = "D:\\Studia\\18L\\SNR\\Projekt\\"

path = folder_path + "SET_C\\"
bounding_boxes = folder_path + "bounding_boxes.txt"

class_count = 50

size_img = 128
std_padding = 2 #USTALONE

cnn_ref = None
base_cnn_ref = None
perceptron = None
cnn = None
svm_cnn = None

add_img = True 
add_img_count = 1#5   #USTALONE
add_img_gauss = True
add_img_gauss_std = 5  #USTALONE
add_type = 3
max_angle = 5   #USTALONE
max_shift = 10  #USTALONE

covered = 1.0
covered_size = 40

test_elements = [0, 3, 7, 12, 13, 14, 28, 29, 47, 54] #Numery wybrane losowo - NIEZMIENNE!!!!!!

# Przetwarzanie danych

In [ ]:
boxes = {}
for row in open(bounding_boxes).read().split('\n'):
    if row == "":
        break
    tmp = row.split()
    tmp[0] = tmp[0].replace("-", "")
    boxes[tmp[0]+".jpg"] = list(map(int, tmp[1:5]))

np.random.seed(1)

test_set = []
training_set = []
data_set_size = 3000.0
k=0.0
for cls, folder in enumerate(os.listdir(path)):
    for index, file in enumerate(os.listdir(path+folder)):
        if k % 10:
            print('Postęp przetwarzania: %0.1f' % (k*100/data_set_size) + "%", end='\r')
        sys.stdout.flush()
        k += 1
        img = cv2.imread(path+folder+"\\"+file, cv2.IMREAD_COLOR)
        bb = boxes[file]
        max_side = max(bb[2], bb[3])

        crop_img = img[bb[1]:bb[1]+bb[3], bb[0]:bb[0]+bb[2]]

        if bb[1]+bb[3] > img.shape[0]:
            bb[3] = crop_img.shape[0]
        if bb[0]+bb[2] > img.shape[1]:
            bb[2] = crop_img.shape[1]
            
        empty_x = math.ceil((max_side - bb[2])/2)
        empty_y = math.ceil((max_side - bb[3])/2)
        add_x = 0
        add_y = 0
        
        if (max_side - bb[2])/2 != round((max_side - bb[2])/2):
            add_x = 1
        if (max_side - bb[3])/2 != round((max_side - bb[3])/2):
            add_y = 1
     
        padding_img = np.zeros((max_side, max_side, 3), dtype=crop_img.dtype)

        padding_img[empty_y:max_side-empty_y + add_y, empty_x:max_side-empty_x + add_x, :] = crop_img
        if(max_side > bb[2]):
            for i in range(empty_x):
                tmp = np.random.normal(0, std_padding, padding_img[:, empty_x, :].shape)  
                padding_img[:, i, :] = padding_img[:, empty_x, :] + tmp  
            
            for i in range(max_side-empty_x + add_x, padding_img.shape[1]):
                padding_img[:, i, :] = padding_img[:, max_side-empty_x + add_x-1, :] + np.random.normal(0, std_padding, padding_img[:, max_side-empty_x + add_x-1, :].shape)
        else:
            for i in range(empty_y):
                padding_img[i, :, :] = padding_img[empty_y, :, :] + np.random.normal(0, std_padding, padding_img[empty_y, :, :].shape)  
            for i in range(max_side-empty_y + add_y, padding_img.shape[0]):
                padding_img[i, :, :] = padding_img[max_side-empty_y + add_y-1, :, :] + np.random.normal(0, std_padding, padding_img[max_side-empty_y + add_y-1, :, :].shape)
            
        
        padding_img[padding_img > 255] = 255
        padding_img[padding_img < 0] = 0
        
        if index in test_elements:
            if add_img:
                continue
                
            resized_image = cv2.resize(padding_img, (size_img, size_img)) 
            
            H0, hogImage0 = feature.hog(resized_image[:, :, 0], orientations=9, pixels_per_cell=(8, 8), cells_per_block=(3, 3), transform_sqrt=True, block_norm="L2-Hys", visualize=True)
            H1, hogImage1 = feature.hog(resized_image[:, :, 1], orientations=9, pixels_per_cell=(8, 8), cells_per_block=(3, 3), transform_sqrt=True, block_norm="L2-Hys", visualize=True)
            H2, hogImage2 = feature.hog(resized_image[:, :, 2], orientations=9, pixels_per_cell=(8, 8), cells_per_block=(3, 3), transform_sqrt=True, block_norm="L2-Hys", visualize=True)
            
            hogImage = np.zeros(resized_image.shape)
            hogImage[:, :, 0] = hogImage0
            hogImage[:, :, 1] = hogImage1
            hogImage[:, :, 2] = hogImage2  
            hog = hogImage.reshape(hogImage.size)
            test_set.append([hog, cls])
        else:
            for i in range(add_img_count + 1):
                resized_image = cv2.resize(padding_img, (size_img, size_img)) 

                if i > 0 and add_type == 1:
                    angle = (np.random.random()*max_angle*2)-max_angle                   
                    resized_image = rotate(resized_image, angle, reshape=False, mode='nearest')
                    if add_img_gauss:
                        resized_image += np.random.normal(0, add_img_gauss,  resized_image.shape).astype(np.uint8)
                if i > 0 and add_type == 2:
                    shift_xy = (np.random.random(2)*max_shift*2)-max_shift
                    resized_image = shift(resized_image, np.append(shift_xy, 0), mode='nearest')
                    if add_img_gauss:
                        resized_image += np.random.normal(0, add_img_gauss,  resized_image.shape).astype(np.uint8)
                if i > 0 and add_type == 3:
                    angle = (np.random.random()*max_angle*2)-max_angle                   
                    shift_xy = (np.random.random(2)*max_shift*2)-max_shift
                    resized_image = shift(resized_image, np.append(shift_xy, 0), mode='nearest')     
                    resized_image = rotate(resized_image, angle, reshape=False, mode='nearest')
                    if add_img_gauss:
                        resized_image += np.random.normal(0, add_img_gauss, resized_image.shape).astype(np.uint8)

                if np.random.random() < covered:
                    x_start = np.random.randint(size_img)
                    y_start = np.random.randint(size_img)
                    resized_image[y_start:y_start+covered_size, x_start:x_start+covered_size, :] = 0

                      
                H0, hogImage0 = feature.hog(resized_image[:, :, 0], orientations=9, pixels_per_cell=(8, 8), cells_per_block=(3, 3), transform_sqrt=True, block_norm="L2-Hys", visualize=True)
                H1, hogImage1 = feature.hog(resized_image[:, :, 1], orientations=9, pixels_per_cell=(8, 8), cells_per_block=(3, 3), transform_sqrt=True, block_norm="L2-Hys", visualize=True)
                H2, hogImage2 = feature.hog(resized_image[:, :, 2], orientations=9, pixels_per_cell=(8, 8), cells_per_block=(3, 3), transform_sqrt=True, block_norm="L2-Hys", visualize=True)

                hogImage = np.zeros(resized_image.shape)
                hogImage[:, :, 0] = hogImage0
                hogImage[:, :, 1] = hogImage1
                hogImage[:, :, 2] = hogImage2  

                hog = hogImage.reshape(hogImage.size)

# DO GENEROWANIA OBRAZKOW DO DOKUMETNACJI
#                 fig, axes = plt.subplots(nrows=1, ncols=4)
# #                 columns = ['Wejściowe', 'Odcięte', 'Normalizacja', 'HOG']
# #                 images = [img, crop_img, resized_image, hogImage]
            
#                 images = [img, crop_img, resized_image, hogImage]
# #                 columns = ['Wejściowe', 'Odcięte', 'Eksperyment', 'HOG']
#                 columns = ['Wejściowe', 'Odcięte', 'Zakryte', 'HOG']

#                 for col in range(len(columns)):
#                         axes[col].axis('off')
#                         axes[col].imshow(images[col])

#                 for ax, col in zip(axes, columns):
#                     ax.set_title(col)

#                 fig.tight_layout()

#                 plt.savefig('fig/'+str(int(k))+"_"+str(i)+'.jpg', dpi = 300,bbox_inches='tight')
#                 plt.show()

                training_set.append([hog, cls])

                if not add_img:
                    break;
test_set = np.array(test_set)
training_set = np.array(training_set)

In [ ]:
training_set.shape

In [ ]:
test_set.shape

In [ ]:
#Zapis danych!

# test_set_file = open('128_128_test.data', 'wb')
# pickle.dump(test_set, test_set_file, protocol= 4) 
# test_set_file.close()

# train_set_file = open('128_128_train.data', 'wb')
# pickle.dump(training_set, train_set_file, protocol= 4)
# train_set_file.close()

# train_set_file = open('128_128_train_rotate_5.data', 'wb')
# pickle.dump(training_set, train_set_file, protocol= 4)
# train_set_file.close()

# train_set_file = open('128_128_train_shift_5.data', 'wb')
# pickle.dump(training_set, train_set_file, protocol= 4)
# train_set_file.close()

# train_set_file = open('128_128_train_rotate_shift_5.data', 'wb')
# pickle.dump(training_set, train_set_file, protocol= 4)
# train_set_file.close()

# train_set_file = open('128_128_train_rotate_shift_gauss_5.data', 'wb')
# pickle.dump(training_set, train_set_file, protocol= 4)
# train_set_file.close()

train_set_file = open('128_128_train_rotate_shift_gauss_covered_15_5.data', 'wb')
pickle.dump(training_set, train_set_file, protocol= 4)
train_set_file.close()

In [6]:
#Odczyt danych!

test_set_file = open('128_128_test.data', 'rb')
test_set = pickle.load(test_set_file)
test_set_file.close()

train_set_file = open('128_128_train.data', 'rb')
training_set = pickle.load(train_set_file)
train_set_file.close()

# train_set_file = open('128_128_train_rotate_5.data', 'rb')
# training_set = pickle.load(train_set_file)
# train_set_file.close()

# train_set_file = open('128_128_train_shift_5.data', 'rb')
# training_set = pickle.load(train_set_file)
# train_set_file.close()

# train_set_file = open('128_128_train_rotate_shift_5.data', 'rb')
# training_set = pickle.load(train_set_file)
# train_set_file.close()

# train_set_file = open('128_128_train_shift_gauss_5.data', 'rb')
# training_set = pickle.load(train_set_file)
# train_set_file.close()

# train_set_file = open('128_128_train_rotate_shift_gauss_5.data', 'rb')
# training_set = pickle.load(train_set_file)
# train_set_file.close()

# train_set_file = open('128_128_train_rotate_shift_gauss_covered_15_5.data', 'rb')
# training_set = pickle.load(train_set_file)
# train_set_file.close()

# Perceptron

In [ ]:
perceptron_hist_1 = []
perceptron_hist_2 = []
perceptron_cmc = []

In [ ]:
layer_size_base = 32
# layers = np.array([2, 4, 5, 8, 10])
# layers = np.array([3, 6, 9, 12])
# layers = np.array([5, 10, 15])
# layers = np.array([10, 20])
layers = np.array([30])

layers = layers * layer_size_base

input_size = training_set[0][0].shape[0]

In [ ]:
if perceptron is not None:
    del perceptron
    perceptron = None 
perceptron = Sequential()
perceptron.add(Dense(layers[0], input_dim=input_size))
perceptron.add(Activation('relu'))
for l in layers[1:len(layers)]:
    perceptron.add(Dense(l))
    perceptron.add(Activation('relu'))
perceptron.add(Dense(class_count))
perceptron.add(Activation('softmax'))

In [ ]:
perceptron.compile(optimizer=SGD(lr=0.005, momentum=0.9),
              loss='categorical_crossentropy',
              metrics=['accuracy', top_k(5)])
perceptron_hist_1.append(perceptron.fit(np.stack(training_set[:, 0], axis = 0), 
          np_utils.to_categorical(training_set[:, 1], num_classes=class_count), 
          epochs=15, 
          batch_size=32,
          verbose = 1))

# perceptron.compile(optimizer=SGD(lr=0.0005, momentum=0.9),
#               loss='categorical_crossentropy',
#               metrics=['accuracy', top_k(5)])
# perceptron_hist_2.append(perceptron.fit(np.stack(training_set[:, 0], axis = 0), 
#           np_utils.to_categorical(training_set[:, 1], num_classes=class_count), 
#           epochs=25, 
#           batch_size=32,
#           verbose = 1))

perceptron_cmc.append(cmc(perceptron, test_set, False, None))
history(perceptron_hist_1[-1], perceptron_hist_2[-1])

In [ ]:
# more_cmc(perceptron_cmc, [5, 4, 3, 2, 1], 'Liczba warstw', 'perceptron_cmc.png')
# more_history(perceptron_hist_1, perceptron_hist_2, 'acc', [5, 4, 3, 2, 1], 'Liczba warstw', 'perceptron_history_1.png')
# more_history(perceptron_hist_1, perceptron_hist_2, 'top_5', [5, 4, 3, 2, 1], 'Liczba warstw', 'perceptron_history_2.png')
# np.savetxt('perceptron_cmc.txt', np.array(perceptron_cmc), fmt = '%0.2f')
# for i, j in zip(perceptron_hist_1, perceptron_hist_2):
#     history(i,j)

# Sieć splotowa - VGG19

In [ ]:
if cnn_ref is not None:
    del cnn_ref
    del base_cnn_ref
    cnn_ref = None 
    base_cnn_ref = None

base_cnn_ref = VGG19(weights='imagenet', input_shape = (size_img, size_img, 3), include_top=False)

x = base_cnn_ref.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
predictions = Dense(class_count, activation='softmax')(x)
cnn_ref = Model(inputs=base_cnn_ref.input, outputs=predictions)

for layer in base_cnn_ref.layers:
    layer.trainable = False

for layer in base_cnn_ref.layers[12:]:
    layer.trainable = True

In [ ]:
cnn_ref.compile(optimizer=SGD(lr=0.005, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy', top_k(5)])
cnn_ref_hist_1 = cnn_ref.fit(np.stack(training_set[:, 0], axis = 0).reshape((training_set.shape[0], size_img, size_img, 3)), 
          np_utils.to_categorical(training_set[:, 1], num_classes=class_count), 
          epochs=15, 
          batch_size=32,
          verbose = 1)

cnn_ref.compile(optimizer=SGD(lr=0.0005, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy', top_k(5)])
cnn_ref_hist_2 = cnn_ref.fit(np.stack(training_set[:, 0], axis = 0).reshape((training_set.shape[0], size_img, size_img, 3)), 
          np_utils.to_categorical(training_set[:, 1], num_classes=class_count), 
          epochs=20, 
          batch_size=32,
          verbose = 1)

cnn_ref_cmc = cmc(cnn_ref, test_set, True, None)
history(cnn_ref_hist_1, cnn_ref_hist_2)

In [ ]:
more_cmc([cnn_ref_cmc], ['VGG19'], 'Nazwa sieci', 'vgg19_cmc.png')
more_history([cnn_ref_hist_1], [cnn_ref_hist_2], 'acc', ['VGG19'], 'Nazwa sieci', 'vgg19_history_1.png')
more_history([cnn_ref_hist_1], [cnn_ref_hist_2], 'top_5', ['VGG19'], 'Nazwa sieci', 'vgg19_history_2.png')
with open('vgg19_cmc.txt', 'a') as file:
    np.savetxt(file, np.array(cnn_ref_cmc), fmt = '%0.5f')

# Sieć splotowa - nasza

In [ ]:
cnn_hist_1 = []
cnn_hist_2 = []
cnn_cmc = []

In [ ]:
# with open('cnn_cmc.txt', 'r') as file:
#     cnn_cmc = np.loadtxt(file)
# # cnn_cmc = cnn_cmc.tolist()
# cnn_cmc = [cnn_cmc]

In [13]:
cnn = Sequential()
cnn.add(Conv2D(1, kernel_size=(5, 5), strides=(1, 1),
                 activation='relu',
                 input_shape=(size_img, size_img, 3)))

In [14]:
cnn.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 124, 124, 1)       76        
Total params: 76
Trainable params: 76
Non-trainable params: 0
_________________________________________________________________


In [ ]:
dropout = True
droput_rate = 0.5

descriptor_size = 1024

if cnn is not None:
    del cnn
    cnn = None 

cnn = Sequential()
cnn.add(Conv2D(128, kernel_size=(3, 3), strides=(1, 1),
                 activation='relu',
                 input_shape=(size_img, size_img, 3)))
cnn.add(MaxPooling2D(pool_size=(2, 2)))

cnn.add(Conv2D(64, (5, 5), activation='relu'))
cnn.add(MaxPooling2D(pool_size=(2, 2)))

cnn.add(Flatten())

cnn.add(Dense(descriptor_size, activation='relu'))

if dropout:
    cnn.add(Dropout(droput_rate))
    
cnn.add(Dense(class_count, activation='softmax'))

In [ ]:
cnn.summary()

In [ ]:
cnn.compile(optimizer=SGD(lr=0.005, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy', top_k(5)])
cnn_hist_1.append(cnn.fit(np.stack(training_set[:, 0], axis = 0).reshape((training_set.shape[0], size_img, size_img, 3)), 
          np_utils.to_categorical(training_set[:, 1], num_classes=class_count), 
          epochs=7, 
          batch_size=32,
          verbose = 1))

cnn.compile(optimizer=SGD(lr=0.0005, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy', top_k(5)])
cnn_hist_2.append(cnn.fit(np.stack(training_set[:, 0], axis = 0).reshape((training_set.shape[0], size_img, size_img, 3)), 
          np_utils.to_categorical(training_set[:, 1], num_classes=class_count), 
          epochs=5, 
          batch_size=32,
          verbose = 1))

cnn_cmc.append(cmc(cnn, test_set, True, None))
history(cnn_hist_1[-1], cnn_hist_2[-1])

In [ ]:
title = 'Pokrycie'
# labels = ['Podstawowy', 'Obrót', 'Translacja', 'Translacja + Obrót', 'Translacja + Obrót + Szum']
# labels = ['Podstawowy (512)', '256', '1024']
labels = ['Nie', 'Tak (15%)']

more_cmc(cnn_cmc, labels, title, 'cnn_cmc_pokrycie.png')
more_history(cnn_hist_1, cnn_hist_2, 'acc', labels[1:], title, 'cnn_pokrycie_history_1.png')
more_history(cnn_hist_1, cnn_hist_2, 'top_5', labels[1:], title, 'cnn_pokrycie_history_2.png')

with open('cnn_cmc_pokrycie.txt', 'w') as file:
    np.savetxt(file, np.array(cnn_cmc), fmt = '%0.5f')
    

# SVM

In [ ]:
# cnn.save('cnn_gauus_shift_rotate_5_1024_dropout_covered_15.h5')
# cnn = load_model('cnn_gauus_shift_rotate_5_1024_dropout_covered_15.h5')

In [ ]:
# layer = -3 #Flatten
layer = -2 #Dense
if dropout:
    layer -= 1
    
svm_cnn = Model(inputs=cnn.input, outputs=cnn.layers[layer].output)
svm_descriptors = svm_cnn.predict(
    np.stack(training_set[:, 0], axis = 0).reshape((training_set.shape[0], size_img, size_img, 3)), 
    batch_size=32,
    verbose = 1)
datetime.datetime.now()
svm_clf = svm.SVC(decision_function_shape='ovr', probability = True, verbose = False)
svm_clf.fit(svm_descriptors, training_set[:, 1].astype(np.int32))
datetime.datetime.now()

In [ ]:
svm_test_descriptors = svm_cnn.predict(
    np.stack(test_set[:, 0], axis = 0).reshape((test_set.shape[0], size_img, size_img, 3)), 
    batch_size=32,
    verbose = 1)

svm_results = svm_clf.predict_proba(svm_test_descriptors)
cnn_cmc.append(cmc(None, test_set, False, svm_results))

In [ ]:
title = 'Metoda klasyfikacji'
labels = ['CNN', 'SVM']

more_cmc(cnn_cmc, labels, title, 'cnn_cmc_svm.png')

with open('cnn_cmc_svm.txt', 'w') as file:
    np.savetxt(file, np.array(cnn_cmc), fmt = '%0.5f')

In [ ]:
# with open('svm.model', 'wb') as f:
#     pickle.dump(svm_clf, f)

In [ ]:
with open('cnc_summary.txt', 'r') as file:
    summary = np.loadtxt(file)
summary = summary.tolist()
title = "Podsumowanie"
labels = ["VGG19", "CNN - Zakrycie (15% obrazów)", "SVM", "CNN - dropout (50%)", "CNN - Rotacja + translacja", "Perceptron - 1 warstwa"]
more_cmc(summary, labels, title, 'summary.png')
